In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm

import time
import config
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
class Product(object):
    def __init__(self, WebElement):
        self.ID = WebElement.find('a')['href'].split('/')[-2]
        self.Name = WebElement.find('a').find('h2').get_text()
    
        self.CPU, self.RAM, self.GPU, self.Display, self.Storage = None, None, None, None, None

        cpu = WebElement.find(class_="cpu")
        ram = WebElement.find(class_="ram")
        gpu = WebElement.find(class_="gpu")
        display = WebElement.find(class_="display")
        storage = WebElement.find(class_="storage")

        if cpu is not None:
            self.CPU = cpu.get_text()
        if ram is not None:
            self.RAM = ram.get_text()
        if gpu is not None:
            self.GPU = gpu.get_text()
        if display is not None:
            self.Display = display.get_text()
        if storage is not None:
            self.Storage = storage.get_text()

        self.Link = WebElement.find('a')['href']
        self.Image = WebElement.find('img')['src']
        self.Price = WebElement.find('span', class_='text-lm-darkBlue').get_text()
        self.Price = int(self.Price[1:])

    def print(self):
        attrs = vars(self)
        print('\n'.join("%s: %s" % item for item in attrs.items() if item[0] != 'WebElement'))

    def extract_to_dataframe(self):
        return pd.DataFrame([vars(self)])

In [3]:
run = 'OTHER'
os.makedirs(f'../Data/{run}', exist_ok=True)
url = config.OTHER

In [4]:
no_page = 1
while(True):
    driver = webdriver.Chrome()
    driver.get(url.format(no_page=no_page))
    time.sleep(10)
    page_source = driver.page_source
    driver.close()

    soup = BeautifulSoup(page_source, 'html.parser')

    list_all = soup.find_all(class_="flex items-center gap-2 list-none border-b py-1 mb-2")
    if len(list_all) == 0:
        break

    df = None

    print(no_page)

    for ele in tqdm(list_all):
        item = Product(ele)

        if df is not None:
            df = pd.concat([df, item.extract_to_dataframe()], ignore_index=True)
        else:
            df = item.extract_to_dataframe()

    df.to_csv(f'../Data/{run}/{no_page}.csv', index=False)
    no_page = no_page + 1

1


  0%|          | 0/1000 [00:00<?, ?it/s]

2


  0%|          | 0/1000 [00:00<?, ?it/s]

3


  0%|          | 0/1000 [00:00<?, ?it/s]

4


  0%|          | 0/545 [00:00<?, ?it/s]